In [ ]:
# display(work_compare.sort_values('work delta'),'all')
# work_compare_dict[0]

In [9]:
from setup import *
students = pd.read_csv('students.csv')
# students = students.set_index('NAME')

work_plan = pd.read_csv('work_tracker.csv')
work_plan['Device ID'] = work_plan['Device ID'].fillna(0).astype(int)
work_plan.drop('work3',axis=1,inplace=True)
work_plan.drop('work4',axis=1,inplace=True)
IDs = work_plan['Device ID']

posters = pd.read_csv('posters.csv')
orals = pd.read_csv('orals.csv')
df = pd.concat([posters,orals])
judge_idx = df['Device ID'].isin(IDs)
earl_rudder_idx = df['Session/Item ID'] == 8909932
valid_idx = judge_idx & ~earl_rudder_idx
scores = df[valid_idx]

unique_idx = ((scores['Question ID'] == 157036) | (scores['Question ID'] == 164187))
unique = scores.loc[unique_idx,:]

work_done = unique.groupby('Device ID')['Session/Item Name'].count().to_frame()
work_done.columns = ['work done']
work_compare_df = pd.merge(work_plan,work_done,left_on='Device ID',right_index=True)
work_compare_df['work delta'] = work_compare_df['work done'] - work_compare_df['work expected']

work_compare_dict = work_compare_df.to_dict(orient='index')
for (key,val) in work_compare_dict.items():
    id = val['Device ID']
    records = unique[unique['Device ID'] == id]['Session/Item Name'].tolist()
    val['records'] = records
work_compare_dict[0]

comments_idx = ((scores['Question ID'] == 150165) | (scores['Question ID'] == 150298))
password_idx = ((scores['Question ID'] == 144457) | (scores['Question ID'] == 144456))
numeric_idx = ~(comments_idx | password_idx)
numeric = scores.loc[numeric_idx].copy()

numeric["Response"] = numeric["Response"].str.partition(':')[0]
good_idx = numeric['Response'].str.isnumeric()
print("%u valid scores and %u invalid scores (listed below)"%(good_idx.sum(),(~good_idx).sum()))
display(numeric[~good_idx],'all')
numeric = numeric.loc[good_idx]
numeric['Response'] = numeric['Response'].astype(float)

R = numeric.groupby('Device ID')['Response']
print("Summary stats for judges before normalization")
display(R.describe())

norm_scores = numeric.copy()
norm_scores['Response'] = R.transform(lambda x: (x-x.mean())/x.std())
print("Summary stats for judges before after normalization.  Note every mean=0 and std=1, as desired.")
display(norm_scores.groupby('Device ID')['Response'].describe())#.unstack())
# print("Rankings of participants using standardized scores")
ranks = norm_scores.groupby('Session/Item Name')['Response'].describe().sort_values(by='mean',ascending=False)
# display(ranks)
students = pd.merge(students,ranks,left_on='NAME',right_index=True,how='outer',indicator=True)

categories = students['CATEGORY'].unique().tolist()
levels = students['STUDENT LEVEL'].unique().tolist()
try:
    levels.remove('Doctoral student')
except:
    pass
levels
categories
for cat in categories:
    for lev in levels:
        stud_idx = ((students['CATEGORY'] == cat) & (students['STUDENT LEVEL'] == lev))
        print(cat, lev)
        display(students.loc[stud_idx,'NAME':].sort_values(by='mean',ascending=False),'all')#where='popup',name='ranks')


Engineering & Computer Science Master's student


,NAME,PRESENTATION TYPE,STUDENT LEVEL,CATEGORY,INSTITUTION,count,mean,std,min,25%,50%,75%,max,_merge
20,"Bedekar, Himani",poster,Master's student,Engineering & Computer Science,Texas A&M University - Kingsville,16.000,0.719,1.072,-1.135,0.110,0.664,1.774,2.046,both
186,"Nwabugwu, Chimezie",poster,Master's student,Engineering & Computer Science,Texas A&M University - Kingsville,16.000,0.336,0.805,-1.135,0.170,0.471,0.874,1.725,both
126,"Kothapally, Akhil Reddy",poster,Master's student,Engineering & Computer Science,Texas A&M University - Corpus Christi,8.000,0.083,1.090,-1.459,-0.651,0.473,0.789,1.442,both
22,"Bell, Duane",poster,Master's student,Engineering & Computer Science,Prairie View A&M University,12.000,0.057,1.111,-2.016,-0.616,0.357,0.831,1.235,both
79,"Ganesan Valli, Sruthi",poster,Master's student,Engineering & Computer Science,Texas A&M University - Kingsville,16.000,-0.055,0.787,-1.758,-0.398,0.151,0.341,1.093,both
13,"Balzen, Hunter",poster,Master's student,Engineering & Computer Science,Texas A&M University - Kingsville,16.000,-0.133,0.893,-1.135,-0.740,-0.398,0.371,1.683,both
200,"Potluri, Sai Pradeep",poster,Master's student,Engineering & Computer Science,Prairie View A&M University,16.000,-0.182,0.954,-1.459,-0.974,-0.123,0.274,1.683,both
3,"Aguirre, Isabella",poster,Master's student,Engineering & Computer Science,Texas A&M University - Kingsville,nan,nan,nan,nan,nan,nan,nan,nan,left_only
4,"Agwuegbo, Chiamaka",poster,Master's student,Engineering & Computer Science,Texas A&M University - Kingsville,nan,nan,nan,nan,nan,nan,nan,nan,left_only
8,"Arsekar, Shubharaj",poster,Master's student,Engineering & Computer Science,Texas A&M University - Corpus Christi,nan,nan,nan,nan,nan,nan,nan,nan,left_only


Engineering & Computer Science Undergraduate student


,NAME,PRESENTATION TYPE,STUDENT LEVEL,CATEGORY,INSTITUTION,count,mean,std,min,25%,50%,75%,max,_merge
181,"Nguyen, Vuong",poster,Undergraduate student,Engineering & Computer Science,Texas A&M University - Corpus Christi,12.000,0.668,0.341,0.249,0.249,0.758,0.899,1.093,both
76,"Flores, Laura",poster,Undergraduate student,Engineering & Computer Science,Texas A&M University - Corpus Christi,16.000,0.296,0.883,-0.941,-0.740,0.616,1.235,1.335,both
163,"Molina, Daniel",poster,Undergraduate student,Engineering & Computer Science,Texas A&M International University,12.000,0.142,0.816,-1.271,-0.235,0.462,0.832,0.832,both
84,"Garcia, Jose",poster,Undergraduate student,Engineering & Computer Science,Texas A&M University - Kingsville,12.000,0.141,0.786,-1.135,-0.092,0.552,0.653,0.848,both
190,"Osei, Michael",poster,Undergraduate student,Engineering & Computer Science,Tarleton State University,16.000,-0.061,1.349,-3.575,-0.585,0.357,0.874,1.292,both
220,"Samayoa, Susana",poster,Undergraduate student,Engineering & Computer Science,Texas A&M University - Kingsville,20.000,-0.206,0.843,-2.497,-0.740,-0.003,0.616,0.664,both
153,"McDougald, Emory",poster,Undergraduate student,Engineering & Computer Science,Tarleton State University,20.000,-0.360,1.067,-3.583,-0.585,-0.380,0.465,0.653,both
21,"Bedwell, Cody",poster,Undergraduate student,Engineering & Computer Science,Tarleton State University,16.000,-0.519,0.948,-2.401,-1.344,-0.486,-0.055,0.989,both
7,"Armah, Esther",poster,Undergraduate student,Engineering & Computer Science,Prairie View A&M University,nan,nan,nan,nan,nan,nan,nan,nan,left_only
26,"Bernier-Vega, Agustin",poster,Undergraduate student,Engineering & Computer Science,Texas A&M University - Kingsville,nan,nan,nan,nan,nan,nan,nan,nan,left_only


Engineering & Computer Science nan


...

,NAME,PRESENTATION TYPE,STUDENT LEVEL,CATEGORY,INSTITUTION,count,mean,std,min,25%,50%,75%,max,_merge


nan Undergraduate student


,NAME,PRESENTATION TYPE,STUDENT LEVEL,CATEGORY,INSTITUTION,count,mean,std,min,25%,50%,75%,max,_merge


nan nan


,NAME,PRESENTATION TYPE,STUDENT LEVEL,CATEGORY,INSTITUTION,count,mean,std,min,25%,50%,75%,max,_merge
